In [1]:
import pandas as pd
import csv
import numpy as np
import random
import numpy as np
from datetime import datetime
import networkx as nx
import time
import math

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier

import pickle

In [2]:
random.seed(16)

## File Paths

In [3]:
train_file = 'train.csv'
test_file = 'test.csv'

## Getting nodes from test data

In [5]:
test_nodes = []

with open(test_file, 'r') as file:
    reader = csv.reader(file)
    
    # Build the graph from the CSV file
    for row in reader:
        test_nodes.extend(row[1:])

test_nodes = set(test_nodes[2:])
print(f'{len(test_nodes)} test nodes loaded...')

3773 test nodes loaded...


## Loading raw data

In [13]:
adjacency_list = {}
high_degree_nodes = {}

with open(train_file, 'r') as file:
    reader = csv.reader(file)
    # Build the graph from the CSV file
    for row in reader:
        node = row[0]
        neighbors = row[1:]
        if len(neighbors) <= 10000:
            adjacency_list[node] = neighbors
        else:
            high_degree_nodes[node] = len(neighbors)
            

print(f'{len(adjacency_list)} nodes and their out-neighbours loaded...')

19639 nodes and their out-neighbours loaded...


## Directed Graph

In [7]:
DiG = nx.DiGraph(adjacency_list)

#### Undirected Graph

In [8]:
G = nx.Graph(adjacency_list)

In [9]:
today_date = datetime.now().strftime("%Y%m%d_%H%M")
nx.write_graphml(DiG, f"data\{today_date}_train_digraph.graphml")

In [10]:
all_nodes = list(DiG.nodes())
#total_num_edges = list(DIG.edges())
#print(total_num_nodes, total_num_edges)
print(f'DiGraph with {len(all_nodes)} nodes and {len(DiG.edges())} edges created')

DiGraph with 1975037 nodes and 10085375 edges created


## Sampling True & False Edges

### True Edge Sampling

In [11]:
def random_walk(graph, start_node, walk_length):
    walk = [start_node]
    
    for _ in range(walk_length):
        neighbors = list(graph.successors(walk[-1]))  # Get neighbors of the last visited node
        if neighbors:
            next_node = random.choice(neighbors)
            walk.append(next_node)
        else:
            break  # If no neighbors, terminate the walk
    
    return walk

def reverse_random_walk(graph, start_node, walk_length):
    walk = [start_node]
    
    for _ in range(walk_length):
        neighbors = list(graph.predecessors(walk[-1]))  # Get neighbors of the last visited node
        if neighbors:
            next_node = random.choice(neighbors)
            walk.append(next_node)
        else:
            break  # If no neighbors, terminate the walk
    
    return walk

In [33]:
walk_length = 10

# Initialize a list to store sampled edges
sampled_edges = set()
num_of_edges_to_sample = 25000

# Perform random walks and sample edges
while len(sampled_edges) < (num_of_edges_to_sample/2):
    for start_node in test_nodes:
        if start_node in DiG.nodes():
            #start_node = random.choice(list(adjacency_list.keys()))
            walk = random_walk(DiG, start_node, walk_length)        
            # Extract edges from the random walk
            edges = [(walk[i], walk[i+1]) for i in range(len(walk)-1)]
            sampled_edges.update(edges)
        if len(sampled_edges) >= (num_of_edges_to_sample/2):
            break

while len(sampled_edges) < (num_of_edges_to_sample):
    for start_node in test_nodes:
        if start_node in DiG.nodes():
            #start_node = random.choice(list(adjacency_list.keys()))
            walk = reverse_random_walk(DiG, start_node, walk_length)        
            # Extract edges from the random walk
            edges = [(walk[i+1], walk[i]) for i in range(len(walk)-1)]
            sampled_edges.update(edges)
        if len(sampled_edges) >= (num_of_edges_to_sample):
            break

In [34]:
true_edges = list(sampled_edges)

In [35]:
# # Sample 25k true edges
# true_edges = random.sample(DIG.edges(), 25000)

In [36]:
print(f'{len(true_edges)} true edges sampled, Some sampled edges are: {true_edges[:3]}')

25003 true edges sampled, Some sampled edges are: [('1712203', '3826797'), ('4473317', '3636653'), ('4441968', '1707083')]


### False Edge Sampling

In [ ]:
# false_edges = []
# total_false_edges_count = 25000
# hard_false_edges_count=15000

# start_time = time.time()  # Get the current time
# duration = 120 * 60  # 120 minutes in seconds

# # Hard Non Edges = Two hops apart from each other
# while len(false_edges) < hard_false_edges_count:
#     source = random.choice(all_nodes)
#     sink = random.choice(all_nodes)
#     if source != sink and (source, sink) not in false_edges:
#         if (source, sink) not in false_edges:
#             try:
#                 shortest_path_length = nx.shortest_path_length(DIG, source=source, target=sink)
#                 if shortest_path_length == 2:
#                     false_edges.append((source, sink))
#             except nx.NetworkXNoPath:
#                 pass
#     if time.time() - start_time > duration:
#         print("120 minutes have elapsed. Breaking the loop.")
#         break

# print('Got hard_false_edges_count: ', len(false_edges))

# hard_false_edges_len = len(false_edges)

# while len(false_edges) <= total_false_edges_count:
#     source = random.choice(all_nodes)
#     sink = random.choice(all_nodes)
#     if source != sink and (source, sink) not in false_edges:
#         if not DIG.has_edge(source, sink):
#             false_edges.append((source, sink))

# len(false_edges)

In [37]:
non_edges = nx.non_edges(DiG)

In [38]:
nodes_in_true_edges = [item for sublist in true_edges for item in sublist]

In [39]:
false_edges = []
for ne in non_edges:
    if len(false_edges) < 25000:
        if (ne[0] in nodes_in_true_edges or ne[1] in nodes_in_true_edges):
            false_edges.append(ne)        
    else:
        break

In [40]:
print(f'{len(false_edges)} false edges sampled, Some sampled edges are: {false_edges[:3]}')

25000 false edges sampled, Some sampled edges are: [('687794', '2712371'), ('687794', '4232790'), ('687794', '352193')]


## Sampled Edges Dataframe

### True Edges Dataframe

In [41]:
true_edges_df = pd.DataFrame(true_edges, columns = ['source', 'sink'])
true_edges_df.head(3)

,source,sink
0,1712203,3826797
1,4473317,3636653
2,4441968,1707083


### Adding Labels column

In [42]:
true_edges_df['label'] = 1
true_edges_df.head(3)

,source,sink,label
0,1712203,3826797,1
1,4473317,3636653,1
2,4441968,1707083,1


### False Edges Dataframe

In [43]:
false_edges_df = pd.DataFrame(false_edges, columns = ['source', 'sink'])
false_edges_df.head(3)

,source,sink
0,687794,2712371
1,687794,4232790
2,687794,352193


In [44]:
false_edges_df['label'] = 0
false_edges_df.head(3)

,source,sink,label
0,687794,2712371,0
1,687794,4232790,0
2,687794,352193,0


### Sampled Edges Dataframe (True and False Edges)

In [45]:
edges_df = pd.concat([true_edges_df, false_edges_df])
edges_df.head()

,source,sink,label
0,1712203,3826797,1
1,4473317,3636653,1
2,4441968,1707083,1
3,382439,2353336,1
4,2234341,2649209,1


In [46]:
edges_df['source'] = edges_df['source'].astype(str)
edges_df['sink'] = edges_df['sink'].astype(str)

In [47]:
edges_df.shape

(50003, 3)

In [48]:
edges_df.label.value_counts()

label
1    25003
0    25000
Name: count, dtype: int64

In [49]:
today_date = datetime.now().strftime("%Y%m%d_%H%M")
edges_df.to_csv(f"data\{today_date}_sampled_edges.csv", index = False)

## Features Creation

In [50]:
#edges_df = pd.read_csv("data/20240303_2235_sampled_edges.csv")
#edges_df.head(3)

In [51]:
# edges_df.label.value_counts()

In [52]:
train_df = edges_df.copy()

In [53]:
def adamic_adar_index(common_neighbours):
    return sum([1/math.log(G.degree(x)) if (G.degree(x) != 0 and G.degree(x) != 1) else 0 for x in common_neighbours])

def ra_index(common_neighbours):
    return sum([1/G.degree(x) if G.degree(x) != 0 else 1 for x in common_neighbours])

def pref_attach(G, edge):
    return G.degree(edge[0])*G.degree(edge[1])

def create_undirected_features(df, G, source_col_name, sink_col_name):
    edges_list = list(zip(df[source_col_name].tolist(), df[sink_col_name].tolist()))
    df['common_neighbours'] = [list(nx.common_neighbors(G, edge[0], edge[1])) for edge in edges_list]
    print('Common neighbours retrieved...')
    df['total_neighbours'] = [len(set(G.neighbors(edge[0])).union(set(G.neighbors(edge[1])))) for edge in edges_list]
    print('Total neighbours is calculated...')
    df['pref_attach'] = [pref_attach(G, x) for x in edges_list]
    print('Preferential attachment is calculated...')
    df['ra_index'] = df['common_neighbours'].apply(ra_index)
    print('Resource allocation index is calculated...')
    df['jaccard_coef'] = df['common_neighbours'].apply(len)/df['total_neighbours']
    print('Jaccard coefficient is calculated...')
    df['aa_index'] = df['common_neighbours'].apply(adamic_adar_index)
    print('Adamic adar index is calculated...')

    df.drop(['common_neighbours', 'total_neighbours'], axis=1, inplace=True)

    return df

In [54]:
def create_directed_degree_features(df, DiG, source_col_name, sink_col_name):
    edges_list = list(zip(df[source_col_name].tolist(), df[sink_col_name].tolist()))
    # Feature 1: Source In-Degree Density
    df['source_in_degree_dens'] = df[source_col_name].apply(lambda x : DiG.in_degree(x)/DiG.degree(x))
    # Feature 2: Sink In-Degree Density
    df['sink_in_degree_dens'] = df[sink_col_name].apply(lambda x : DiG.in_degree(x)/DiG.degree(x))
    # Feature 3: Source Out-Degree Density
    df['source_out_degree_dens'] = df[source_col_name].apply(lambda x : DiG.out_degree(x)/DiG.degree(x))
    # Feature 4: Sink Out-Degree Density
    df['sink_out_degree_dens'] = df[sink_col_name].apply(lambda x : DiG.out_degree(x)/DiG.degree(x))
    # Feature 5: Source Bi-Degree Density
    df['source_bi_degree_dens'] = df[source_col_name].apply(lambda x : len(set(DiG.predecessors(x)).intersection(set(DiG.successors(x))))/DiG.degree(x))
    # Feature 6: Sink Bi-Degree Density
    df['sink_bi_degree_dens'] = df[sink_col_name].apply(lambda x : len(set(DiG.predecessors(x)).intersection(set(DiG.successors(x))))/DiG.degree(x))
    # Feature 7: Common In-neighbours
    df['common_in_neighbours'] = [len(set(DiG.predecessors(source)).intersection(set(DiG.predecessors(sink)))) for source, sink in edges_list]
    # Feature 8: Common out-neighbours
    df['common_out_neighbours'] = [len(set(DiG.successors(source)).intersection(set(DiG.successors(sink)))) for source, sink in edges_list]
    # Feature 9: Total In-neighbours
    df['total_in_neighbours'] = [len(set(DiG.predecessors(source)).union(set(DiG.predecessors(sink)))) for source, sink in edges_list]
    # Feature 10: Total out-neighbours
    df['total_out_neighbours'] = [len(set(DiG.successors(source)).union(set(DiG.successors(sink)))) for source, sink in edges_list]
    
    return df

In [55]:
train_df = create_undirected_features(train_df, G, 'source', 'sink')
train_df.head()

Common neighbours retrieved...
Total neighbours is calculated...
Preferential attachment is calculated...
Resource allocation index is calculated...
Jaccard coefficient is calculated...
Adamic adar index is calculated...


,source,sink,label,pref_attach,ra_index,jaccard_coef,aa_index
0,1712203,3826797,1,181500,0.001983,0.002020,0.279720
1,4473317,3636653,1,2752,0.000000,0.000000,0.000000
2,4441968,1707083,1,134366,0.874667,0.034346,6.811151
3,382439,2353336,1,7354527,1.509140,0.059947,53.384630
4,2234341,2649209,1,678,0.000000,0.000000,0.000000


In [ ]:
train_df = create_directed_degree_features(train_df, DiG, 'source', 'sink')
train_df.head()

In [56]:
today_date = datetime.now().strftime("%Y%m%d_%H%M")
train_df.to_csv(f"data\{today_date}_train.csv", index = False)

## Model

In [ ]:
# train_df = pd.read_csv("data/20240303_2357_train_directed.csv")
# train_df.head(3)

In [ ]:
# train_df.label.value_counts()

### Defining X and y

In [57]:
X = train_df.drop(['source','sink','label'], axis=1)
y = train_df['label']

### Scaling the features

In [58]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [59]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 16)
rf.fit(X, y)

# Step 4: Get feature importances
feature_importances = rf.feature_importances_

# Step 5: Analyze results
for i, importance in enumerate(feature_importances):
    print(f"Feature {i+1} importance: {importance}")

Feature 1 importance: 0.26367897366368004
Feature 2 importance: 0.3009162263879774
Feature 3 importance: 0.17604573187681247
Feature 4 importance: 0.25935906807153025


### Initialize Classifier

In [64]:
#model = LogisticRegression(C=1, class_weight = None)
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.01, random_state=16)

### K-Fold Validation

In [65]:
# Specify the number of folds (k)
num_folds = 5

# Create a KFold object
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1)

# Use cross_val_score for cross-validation
cv_results = cross_val_score(model, X, y, cv=kf, scoring='roc_auc')

# Print the cross-validation results
print("Cross-Validation Results:", cv_results)
print("Mean AUC:", cv_results.mean())

Cross-Validation Results: [0.90962242 0.90385635 0.90901186 0.9099935  0.91071136]
Mean AUC: 0.908639096375125


### Fitting the model

In [66]:
model.fit(X,y)

GradientBoostingClassifier(learning_rate=0.01, random_state=16)

### Model interpretation

In [68]:
#model.coef_

In [69]:
today_date = datetime.now().strftime("%Y%m%d_%H%M")
with open(f"data\{today_date}_scaler.pkl", "wb") as f:
    loaded_model = pickle.dump(scaler, f)

In [70]:
with open(f"data\{today_date}_model.pkl", "wb") as f:
    loaded_model = pickle.dump(model, f)

## Prediction

In [ ]:
# DiG = nx.read_graphml("data/20240303_2343_sampled_digraph.graphml", node_type = int)
# print(len(G.edges()))

In [ ]:
# with open("data/20240303_2358_model.pkl", "rb") as f:
#     model = pickle.load(f)

In [ ]:
# model.coef_

In [ ]:
# with open("data/20240303_2358_scaler.pkl", "rb") as f:
#     scaler = pickle.load(f)

### Load Test Data

In [96]:
test_df = pd.read_csv('test.csv')
test_df.head(3)

,Id,From,To
0,1,3360982,4457271
1,2,4761876,4698439
2,3,4198430,3615486


In [97]:
test_df['From'] = test_df['From'].astype(str)
test_df['To'] = test_df['To'].astype(str)

In [98]:
high_degree_test_edges_index = test_df[test_df['From'].apply(lambda x : True if x not in G.nodes() else False) | test_df['To'].apply(lambda x : True if x not in G.nodes() else False)].index
test_df = test_df[test_df['From'].apply(lambda x : True if x in G.nodes() else False) & test_df['To'].apply(lambda x : True if x in G.nodes() else False)]

In [99]:
#test_df = create_directed_degree_features(test_df, DiG, 'From', 'To')
test_df = create_undirected_features(test_df, G, 'From', 'To')
test_df.head()

Common neighbours retrieved...
Total neighbours is calculated...
Preferential attachment is calculated...
Resource allocation index is calculated...
Jaccard coefficient is calculated...
Adamic adar index is calculated...


,Id,From,To,pref_attach,ra_index,jaccard_coef,aa_index
0,1,3360982,4457271,536490,4.526118,0.035796,23.561230
1,2,4761876,4698439,1360,0.000000,0.000000,0.000000
2,3,4198430,3615486,126792,0.000000,0.000000,0.000000
3,4,2945770,747948,20384,0.001357,0.004320,0.260033
4,5,3950088,3360335,185087,0.027880,0.021978,4.381721


In [101]:
test_df.shape

(1965, 7)

In [102]:
X_test = scaler.transform(test_df.drop(['From','To','Id'], axis=1))

In [127]:
output_df = test_df.copy()
output_df = output_df[['Id']]

In [128]:
output_df['Predictions'] = np.transpose(model.predict_proba(X_test))[1]

In [129]:
pd.Series(model.predict(X_test)).value_counts()

0    1055
1     910
Name: count, dtype: int64

In [130]:
model.predict(X_test)

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [131]:
output_df.head()

,Id,Predictions
0,1,0.812575
1,2,0.566393
2,3,0.253039
3,4,0.405228
4,5,0.480960


In [132]:
for i in high_degree_test_edges_index:
    output_df.loc[i] = [i+1, 0.8]

In [133]:
output_df = output_df.sort_index()

In [134]:
today_date = datetime.now().strftime("%Y%m%d_%H%M")
output_df.to_csv(f'submissions\{today_date}_submission.csv', index = False)

## Playground

In [ ]:
# def extract_features(graph, node1, node2):
#     # Common Friends
#     common_friends = len(set(graph.successors(node1)).intersection(set(graph.successors(node2))))
    
#     # Total Friends
#     total_friends = len(set(graph.successors(node1)).union(set(graph.successors(node2))))
    
#     # Resource Allocation Index
#     ra_index = sum(1 / graph.out_degree(neighbor) for neighbor in set(graph.successors(node1)).intersection(set(graph.successors(node2))))
    
#     # Jaccard Coefficient
#     jaccard_coef = nx.jaccard_coefficient(graph, [(node1, node2)])[0][2]
    
#     # Adamic-Adar Index
#     aa_index = sum(1 / log(graph.out_degree(neighbor)) for neighbor in set(graph.successors(node1)).intersection(set(graph.successors(node2))))
    
#     # Preferential Attachment
#     pref_attach = graph.out_degree(node1) * graph.out_degree(node2)
    
#     # Katz Centrality
#     try:
#         katz_cent = nx.katz_centrality(graph, alpha=0.1, beta=0.1)[node1] * nx.katz_centrality(graph, alpha=0.1, beta=0.1)[node2]
#     except nx.NetworkXError:
#         katz_cent = 0  # Handle the case when one of the nodes is not in the largest connected component
        
#     return common_friends, total_friends, ra_index, jaccard_coef, aa_index, pref_attach, katz_cent

# # Example usage:
# # Assuming you have a NetworkX DiGraph named 'graph'
# node1 = 1
# node2 = 2
# features = extract_features(graph, node1, node2)
# print("Common Friends:", features[0])
# print("Total Friends:", features[1])
# print("Resource Allocation Index:", features[2])
# print("Jaccard Coefficient:", features[3])
# print("Adamic-Adar Index:", features[4])
# print("Preferential Attachment:", features[5])
# print("Katz Centrality:", features[6])

In [ ]:
# # Resource Allocation Index (RA Index)
# ra_index = list(nx.resource_allocation_index(DIG, [(node1, node2)]))

# # Jaccard Coefficient
# jaccard_coef = list(nx.jaccard_coefficient(DIG, [(node1, node2)]))

# # Adamic-Adar Index
# aa_index = list(nx.adamic_adar_index(DIG, [(node1, node2)]))

# # Preferential Attachment
# pref_attach = list(nx.preferential_attachment(DIG, [(node1, node2)]))

# # Katz Centrality
# katz_cent = nx.katz_centrality(DIG)

# # Extract Katz Centrality for the given pair of vertices
# katz_cent_node1 = katz_cent[node1]
# katz_cent_node2 = katz_cent[node2]

In [ ]:
pr = nx.pagerank(DiG)

In [ ]:
train_df['source'].apply(lambda x : pr[x])

In [ ]:
sorted_dict = dict(sorted(pr.items(), key=lambda x: x[1], reverse = True))

In [ ]:
sorted_dict

In [ ]:
test_df['From'].apply(lambda x : DiG.out_degree(x)).sort_values(ascending=False)

In [ ]:
test_edges_list = list(zip(test_df['From'].tolist(), test_df['To'].tolist()))

In [ ]:
test_edges_list

In [ ]:
sampled_santhosh = pd.read_csv('20240303_2235_sampled_edges.csv')
sampled_santhosh['source'] = sampled_santhosh['source'].astype(str)
sampled_santhosh['sink'] = sampled_santhosh['sink'].astype(str)
sampled_santhosh.head()

In [ ]:
sample_edges_list = list(zip(sampled_santhosh['source'].tolist(),sampled_santhosh['sink'].tolist()))

In [ ]:
sample_edges_list

In [ ]:
set(test_edges_list).intersection(set(sample_edges_list))

In [ ]:
set(test_edges_list).intersection(set(DiG.edges()))

In [ ]:
list(DiG.edges())[:5]

In [ ]:
test_edges_list

In [ ]:
len(set(test_nodes))

In [ ]:
sample_nodes = sampled_santhosh['source'].tolist() + sampled_santhosh['sink'].tolist()

In [ ]:
set(test_nodes)

In [ ]:
len(set(sample_nodes).intersection(set(test_nodes)))

In [ ]:
train_nodes = train_df['source'].tolist() + train_df['sink'].tolist()

In [ ]:
len(set(train_nodes).intersection(set(test_nodes)))

In [ ]:
test_node_degree = [G.degree(x) for x in test_nodes]
train_node_degree = [G.degree(x) for x in train_nodes]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(sorted(test_node_degree, reverse = True), bins=50, edgecolor='black', color='skyblue')
plt.show()

In [ ]:
count = 0
for x in test_node_degree:
    if x > 10000:
        count += 1
print(count)

In [ ]:
len(test_node_degree)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(sorted(test_df['pref_attach'], reverse = True), bins=50, edgecolor='black', color='skyblue')
plt.show()

In [ ]:
sorted(train_df['pref_attach'], reverse = False)

In [ ]:
sum([1,1])